# Orange brix Analysis

## Import libraries

In [167]:
# !pip install lightgbm
# !pip install xgboost

In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from enum import Enum
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error

import os

In [169]:
plt.style.use("fivethirtyeight")

## Settings

In [284]:
GENERATE_PLOTS: bool = False

TRAIN_MODEL_1 : bool = False
TRAIN_MODEL_2 : bool = False
TRAIN_MODEL_3 : bool = False
TRAIN_MODEL_4 : bool = False
TRAIN_MODEL_5 : bool = False

## Create Classes

In [171]:
class Utilities:
    def __init__(self):
        pass

    def get_categorical_numeric_cols(self, dataframe: pd.DataFrame):
        if len(dataframe) == 0:
            raise Exception("No records found!")

        numeric_cols = []
        categorical_cols = []

        for col in dataframe.columns:
            if isinstance(dataframe[col][0], bool) or not isinstance(
                dataframe[col][0], (int, float, np.int8, np.int16, np.int32, np.int64, np.float16, np.float32, np.float64)
            ):
                categorical_cols.append(col)
            else:
                numeric_cols.append(col)

        return (categorical_cols, numeric_cols)

In [172]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    RobustScaler,
    StandardScaler,
)


class ScalerType(Enum):
    Raw_ = None
    MaxAbsScaler_ = MaxAbsScaler
    MinMaxScaler_ = MinMaxScaler
    Normalizer_ = Normalizer
    RobustScaler_ = RobustScaler
    StandardScaler_ = StandardScaler

In [173]:
class Dataset(Utilities):
    def __init__(
        self,
        name: str,
        dataframe: pd.DataFrame,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):
        self.name: str = name
        self.dataframe: pd.DataFrame = dataframe
        self.categorical_cols: list[str] = []
        self.numeric_cols: list[str] = []
        self.scaler_type: ScalerType = scaler_type

        self.categorical_cols, self.numeric_cols = self.get_categorical_numeric_cols(
            self.dataframe
        )

    def get_name(self) -> str:
        return self.name

    def get_dataframe(self) -> pd.DataFrame:
        return self.dataframe

    def detect_categorical_numeric_cols(self):
        if len(self.dataframe) == 0:
            raise Exception("No records found!")

        self.numeric_cols = []
        self.categorical_cols = []

        for col in self.dataframe.columns:
            try:
                float(self.dataframe[col][0])
                self.numeric_cols.append(col)
            except:
                self.categorical_cols.append(col)

    def get_numeric_cols(self) -> list[str]:
        return self.numeric_cols

    def get_numeric_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.numeric_cols]

    def get_categorical_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.categorical_cols]

    def get_scaler_type(self) -> ScalerType:
        return self.scaler_type

    def concat_dataframe(self, additional_dataframe: pd.DataFrame):
        if len(additional_dataframe) == len(self.dataframe):
            self.dataframe = pd.concat([self.dataframe, additional_dataframe], axis=1)
            self.detect_categorical_numeric_cols()

    def get_one_hot_vectorized_dataset(
        self, categorical_col: str, print_name: bool = True
    ):
        if categorical_col in self.categorical_cols:
            dummies_dataframe = pd.get_dummies(self.dataframe[categorical_col])
            dummies_dataframe = pd.concat([dummies_dataframe, self.dataframe], axis=1)
            new_dataset_name = self.name + "_ohv_" + categorical_col
            if print_name:
                print(new_dataset_name)
            return Dataset(
                name=new_dataset_name,
                dataframe=dummies_dataframe,
                scaler_type=self.scaler_type,
            )
        elif categorical_col in self.numeric_cols:
            print("Records for input column name are not categorical!")
        else:
            print("Input column name does not exists!")

In [174]:
class ModelType(Enum):
    LINEAR_REGRESSION: dict = {}
    SVR: dict = {
        "kernel": "rbf", # imp | linear, poly, rbf
        "degree": 3,
        "gamma": "scale",
        "coef0": 0,
        "tol": 1e-3,
        "C": 1.0, # imp tune from 0.1 - 2 step 0.1
        "epsilon": 0.1,
    }
    DECISION_TREE: dict = {
        "criterion": "squared_error",
        "splitter": "best",
        "max_depth": None, # imp tune 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": None, # int 2->5 {“sqrt”, “log2”}, default=None

        "random_state": 42,
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0,
    }
    RANDOM_FOREST: dict = {
        "n_estimators": 100, # int, 100-1000 (step 100)
        "criterion": "squared_error",
        "max_depth": None, # imp tune 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": 1.0, # int 2->5 {“sqrt”, “log2”}, default=None
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0.0,
    }
    LIGHT_GBM: dict = {
        "boosting_type": "gbdt",
        "num_leaves": 31,
        "max_depth": -1,
        "learning_rate": 0.1,
        "n_estimator": 100, # int, 100->1000 (step 100)
        "verbosity": -1,
    }
    XGB: dict = {
        "objective": "reg:linear",
        "n_estimators": 10, # int, 100-1000 (step 100)
        "seed": 123,
        "verbosity": 0,
    }

In [175]:
class Result:
    def __init__(
        self,
        dataset_name: str,
        model_name: str,
        model_type: ModelType,
        scaler_type: str,
        r2: float,
        mse: float,
    ):
        self.result = [dataset_name, model_name, model_type, scaler_type, r2, mse]

    def get_result(self):
        return self.result

In [176]:
class ResultFrame:
    def __init__(self):
        self.result_frame: pd.DataFrame = pd.DataFrame()
        self.count = 0

        self.start_up()

    def start_up(self):
        self.result_frame["dataset_name"] = []
        self.result_frame["model_name"] = []
        self.result_frame["model_type"] = []
        self.result_frame["scaler_type"] = []
        self.result_frame["r2"] = []
        self.result_frame["mse"] = []

    def add_result(self, new_result: Result):
        self.result_frame.loc[self.count] = new_result.result
        self.count += 1

    def display_result(self, display_rows: int = 30):
        self.result_frame.sort_values(["r2", "mse"], ascending=False, inplace=True)
        if display_rows < 0:
            display_rows = 30
        display(self.result_frame.head(display_rows))

    def save_result(self, filename, overwrite=False):
        if self.result_frame.shape[0] == 0:
            print("Result has no records!")
        else:
            file_exists = os.path.isfile(os.path.join(os.getcwd(), filename))
            if file_exists and not overwrite:
                print("Error: CANNOT SAVE FILE.")
                print("A file with the same name already exists.")
                print(
                    "Set overwrite to True to overwrite existed file or change file name!"
                )
            else:
                self.result_frame.to_excel(filename)

In [177]:
class Model(Utilities):
    def __init__(
        self,
        model_type: ModelType,
        model_name: str,
        dataset_name: str,
        x_cols: list[str],
        y_cols: list[str],
        dataset: Dataset,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):

        if len(x_cols) == 0 or len(y_cols) == 0:
            raise Exception("Data columns are not specified!")

        self.model_type: ModelType = model_type
        self.name: str = model_name
        self.dataset_name: str = dataset_name
        self.scaler_type: ScalerType = scaler_type
        self.scaler = self.get_scaler(self.scaler_type)  # could be None
        self.x_dataset: Dataset = None
        self.y_dataset: Dataset = None
        self.inner_model = None

        self.preprocess_data(x_cols, y_cols, dataset)

    # Extract data from x_cols and y_cols
    # Redetect categorical and numeric columns
    def preprocess_data(self, x_cols: list[str], y_cols: list[str], dataset: Dataset):
        x_dataset_new_name = dataset.get_name() + "_x"
        y_dataset_new_name = dataset.get_name() + "_y"
        scaler_type = dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=x_dataset_new_name,
            dataframe=dataset.get_dataframe()[x_cols],
            scaler_type=scaler_type,
        )
        self.y_dataset = Dataset(
            name=y_dataset_new_name,
            dataframe=dataset.get_dataframe()[y_cols],
            scaler_type=scaler_type,
        )

        self.process_x_dataset()

    # arrange [numeric_cols, categorical_cols]
    def process_x_dataset(self):
        new_name = self.x_dataset.get_name()
        numeric_dataframe = self.x_dataset.get_numeric_dataframe()
        categorical_dataframe = self.x_dataset.get_categorical_dataframe()
        scaler_type = self.x_dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=new_name,
            dataframe=pd.concat([numeric_dataframe, categorical_dataframe], axis=1),
            scaler_type=scaler_type,
        )

    def get_x_list_index_numeric_cols(self):
        return list(range(len(self.x_dataset.get_numeric_cols())))

    def fit_scaler_and_scale_dataframe(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            self.scaler.fit(X_numeric)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X

        return transformed_X

    def get_x_dataframe(self):
        return self.x_dataset.get_dataframe()

    def get_y_dataframe(self):
        return self.y_dataset.get_dataframe()

    def get_scaler(self, scaler_type: ScalerType):
        if scaler_type != ScalerType.Raw_:
            return scaler_type.value()

    def fit(self, X=None, Y=None):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())

        model_family = self.model_type.__str__().split(".")[-1]
        params = self.model_type.value

        Y = np.squeeze(Y)

        # need to scale
        transformed_X = self.fit_scaler_and_scale_dataframe(X)

        if model_family == "SVR":
            self.model = SVR(**params).fit(transformed_X, Y)
        elif model_family == "LINEAR_REGRESSION":
            self.model = LinearRegression(**params).fit(transformed_X, Y)
        elif model_family == "DECISION_TREE":
            self.model = DecisionTreeRegressor(**params).fit(transformed_X, Y)
        elif model_family == "RANDOM_FOREST":
            self.model = RandomForestRegressor(**params).fit(transformed_X, Y)
        elif model_family == "LIGHT_GBM":
            self.model = LGBMRegressor(**params).fit(transformed_X, Y)
        elif model_family == "XGB":
            self.model = XGBRegressor(**params).fit(transformed_X, Y)
        else:
            raise ValueError("Unknown model family!")

    def predict(self, X):
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X
        return self.model.predict(transformed_X)

    def evaluate(self, X=None, Y=None, method="LOOCV"):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())
        if method == "LOOCV":
            preds = [0] * len(Y)
            total = len(Y)
            kf = KFold(n_splits=total)
            kf.get_n_splits(Y)

        with tqdm(total=total) as pbar:
            for i, (train_index, valid_index) in enumerate(kf.split(X, Y)):
                X_train = X[train_index]
                Y_train = Y[train_index]
                X_valid = X[valid_index]
                Y_valid = Y[valid_index]
                self.fit(X_train, Y_train)

                Y_valid_pred = self.predict(X_valid)
                for j in range(len(valid_index)):
                    index = valid_index[j]
                    value = Y_valid_pred[j]
                    preds[index] = value
                pbar.update(1)

        r2 = r2_score(Y, preds)
        mse = mean_squared_error(Y, preds)

        # Compile result
        return Result(
            dataset_name=self.dataset_name,
            model_name=self.name,
            model_type=self.model_type,
            scaler_type=self.scaler_type,
            r2=r2,
            mse=mse,
        )

    def save(self, filename: str):
        pass

    def load(self, filename: str):
        pass

    def set_params(self, params: dict):
        pass

## Create main variables

In [178]:
list_dataset: dict[str, Dataset] = dict()
list_model: dict = dict()
result_frame: ResultFrame = ResultFrame()
model_count = 1

## Create functions

In [179]:
def list_all_dataset_names():
    return list_dataset.keys()

In [180]:
def list_all_model_names():
    return list_model.keys()

In [181]:
def plot_along_dataset(
    dataset: Dataset,
    nrow_subplot: int,
    ncol_subplot: int,
    figsize: tuple[int, int],
):
    numeric_df = dataset.get_dataframe()[dataset.numeric_cols]
    x = np.arange(len(numeric_df))
    figs, axes = plt.subplots(nrow_subplot, ncol_subplot, figsize=figsize)
    for i in range(nrow_subplot):
        for j in range(ncol_subplot):
            ax = axes[i][j]
            index = i * ncol_subplot + j
            ax.plot(x, numeric_df.iloc[:, index])
            ax.set_title(f"{numeric_df.columns[index]} along dataframe", size=15)
            ax.xaxis.set_tick_params(labelsize=12)
            ax.xaxis.set_ticks(np.arange(0, len(numeric_df), 10))
            ax.yaxis.set_tick_params(labelsize=12)

In [182]:
def plot_hist_kde_box_all_columns(dataset: Dataset):

    number_of_numeric_col = len(dataset.numeric_cols)
    figs, axes = plt.subplots(
        number_of_numeric_col, 3, figsize=(30, number_of_numeric_col * 10)
    )

    numeric_dataframe = dataset.get_dataframe()[dataset.numeric_cols]

    for i in range(number_of_numeric_col):
        axes[i][0].set_title(f"{numeric_dataframe.columns[i]} histogram", size=20)
        axes[i][0].yaxis.set_tick_params(labelsize=15)
        axes[i][0].xaxis.set_tick_params(labelsize=15)
        sns.histplot(ax=axes[i][0], data=numeric_dataframe.iloc[:, i])

        axes[i][1].set_title(f"{numeric_dataframe.columns[i]} KDE", size=20)
        axes[i][1].xaxis.set_tick_params(labelsize=15)
        axes[i][1].yaxis.set_tick_params(labelsize=15)
        sns.kdeplot(ax=axes[i][1], data=numeric_dataframe.iloc[:, i])

        axes[i][2].set_title(f"{numeric_dataframe.columns[i]} Boxplot", size=20)
        axes[i][2].xaxis.set_tick_params(labelsize=15)
        axes[i][2].yaxis.set_tick_params(labelsize=15)
        sns.boxplot(ax=axes[i][2], data=numeric_dataframe.iloc[:, i])

In [183]:
def plot_all_columns_group_by_a_categorical_col(dataset: Dataset, categorical_col: str):

    dataframe = dataset.get_dataframe()
    numeric_dataframe = dataframe[dataset.numeric_cols]
    number_of_numeric_col = len(dataset.numeric_cols)
    figs, axes = plt.subplots(
        number_of_numeric_col, 2, figsize=(20, number_of_numeric_col * 10)
    )
    groups = dataframe[categorical_col].unique()

    for i in range(number_of_numeric_col):
        axes[i][0].set_title(f"{numeric_dataframe.columns[i]} KDE", size=20)
        axes[i][0].xaxis.set_tick_params(labelsize=15)
        axes[i][0].yaxis.set_tick_params(labelsize=15)
        for group in groups:
            sns.kdeplot(
                ax=axes[i][0],
                data=numeric_dataframe[dataframe[categorical_col] == group].iloc[:, i],
                label=f"Type {group}",
            )
        axes[i][0].legend()

        axes[i][1].set_title(f"{numeric_dataframe.columns[i]} Boxplot", size=20)
        axes[i][1].xaxis.set_tick_params(labelsize=15)
        axes[i][1].yaxis.set_tick_params(labelsize=15)
        sns.boxplot(
            ax=axes[i][1],
            hue="type",
            y=numeric_dataframe.columns[i],
            data=dataframe,
        )

In [184]:
def divide_dataframe_with_categorical_col(
    dataset: Dataset, categorical_col: str
) -> dict[str, pd.DataFrame]:
    dataframe = dataset.get_dataframe()
    groups = dataframe[categorical_col].unique()

    result = {}
    for group in groups:
        result[group] = dataframe[dataframe[categorical_col] == group]

    return result

In [185]:
# def calculate_p_values_between_partitions(dataframes, columns_to_compare):
#     p_values = {}
#     for key1, df1 in dataframes.items():
#         for key2, df2 in dataframes.items():
#             if key1 != key2:
#                 for col in columns_to_compare:
#                     if col in df1 and col in df2:
#                         t_stat, p_val = stats.ttest_ind(df1[col], df2[col])
#                         p_values[f"{col}_{key1}-{col}_{key2}"] = p_val
#     return p_values

In [186]:
# import statsmodels.api as sm
# from sklearn.metrics import mean_squared_error
# from typing import Optional, Callable


# def summary_linear_model(x: list, y, dataset, transformer: Optional[Callable] = None):
#     x_ = sm.add_constant(dataset[x])
#     y_ = dataset[y]
#     results = sm.OLS(y_, x_).fit()
#     Intercept = results.params.iloc[0]
#     Slope = results.params.iloc[1:]
#     P_values = results.pvalues[1:]
#     R_squared = results.rsquared
#     MSE = mean_squared_error(y_, results.predict())
#     print("Intercept\n", Intercept, end="\n\n")
#     print("Slope\n", Slope, end="\n\n")
#     print("P_values\n", P_values, end="\n\n")
#     print("R_squared\n", R_squared, end="\n\n")
#     print("MSE\n", MSE, end="\n\n")
#     return {
#         "Intercept": Intercept,
#         "Slope": Slope,
#         "P_values": P_values,
#         "R_squared": R_squared,
#         "MSE": MSE,
#     }

In [187]:
# intercept_a = model_no_scale["Slope"]["A"] + model_no_scale["Intercept"]
# intercept_b = model_no_scale["Slope"]["B"] + model_no_scale["Intercept"]
# intercept_c = model_no_scale["Slope"]["C"] + model_no_scale["Intercept"]

# x = np.arange(model_dataset["mass"].min(), model_dataset["mass"].max())
# ya = x * model_no_scale["Slope"]["mass"] + intercept_a
# yb = x * model_no_scale["Slope"]["mass"] + intercept_b
# yc = x * model_no_scale["Slope"]["mass"] + intercept_c
# plt.plot(x, ya, linewidth=2, label="type_A")
# plt.plot(x, yb, linewidth=2, label="type_B")
# plt.plot(x, yc, linewidth=2, label="type_C")
# plt.legend()

## Import data

In [188]:
raw_df = pd.read_excel("orange_data.xlsx", sheet_name=0)

In [189]:
raw_df.head(10)

,Name,p,C NaOH,Khối lượng(g),Đường kính ngang(mm),Đường kính dọc(mm),Độ đường (Brix %),TA (%),Brix:TA,BrmTA
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748


In [190]:
raw_df.columns

Index(['Name', 'p', 'C NaOH', 'Khối lượng(g)', 'Đường kính ngang(mm)',
       'Đường kính dọc(mm)', 'Độ đường (Brix %)', 'TA (%)', 'Brix:TA',
       'BrmTA'],
      dtype='object')

In [191]:
raw_df.shape

(81, 10)

In [192]:
new_column_names = {
    "Name": "name",
    "p": "v_naoh",
    "C NaOH": "c_naoh",
    "Khối lượng(g)": "mass",
    "Đường kính ngang(mm)": "h_diameter",
    "Đường kính dọc(mm)": "v_diameter",
    "Độ đường (Brix %)": "brix",
    "TA (%)": "ta",
    "Brix:TA": "brix_ta",
    "BrmTA": "brm_ta",
}

std_col_name_df = raw_df.rename(columns=new_column_names)

In [193]:
std_col_name_df.head(10)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748


In [194]:
std_col_name_dataset = Dataset("orange_std_col_name", std_col_name_df, ScalerType.Raw_)

In [195]:
print(std_col_name_dataset.categorical_cols)
print(std_col_name_dataset.numeric_cols)
std_col_name_dataset.dataframe

['name']
['v_naoh', 'c_naoh', 'mass', 'h_diameter', 'v_diameter', 'brix', 'ta', 'brix_ta', 'brm_ta']


,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
...,...,...,...,...,...,...,...,...,...,...
76,C24,1.60,0.4016,177,225,216,12.5,4.115040,3.037638,8.384960
77,C25,1.60,0.4016,173,216,220,11.1,4.115040,2.697422,6.984960
78,C26,0.95,0.4016,126,195,197,12.8,2.443305,5.238806,10.356695
79,C28,1.70,0.4016,181,216,232,11.8,4.372230,2.698852,7.427770


In [196]:
list_dataset["orange_std_col_name"] = std_col_name_dataset

In [197]:
list_dataset["orange_std_col_name"].get_dataframe().head(10)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748


In [198]:
if GENERATE_PLOTS:
    plot_along_dataset(list_dataset["std_col_name"], 3, 3, (12, 12))

#### From the shape of lines in graph "mass", "h_diameter", "v_diameter" and "brix", we can see the lines could be divided into 3 levels along dataset. We assume that there are 3 types of data in the dataset. We could double check the name / label of dataset.

In [199]:
list_dataset["orange_std_col_name"].get_dataframe().columns

Index(['name', 'v_naoh', 'c_naoh', 'mass', 'h_diameter', 'v_diameter', 'brix',
       'ta', 'brix_ta', 'brm_ta'],
      dtype='object')

In [200]:
print(list_dataset["orange_std_col_name"].get_dataframe()["name"].to_list())

['A1', 'A2', 'A3', 'A4', 'A5', 'A7', 'A8', 'A9', 'A10', 'A11', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C8', 'C9', 'C10', 'C11', 'C12', 'C14', 'C15', 'C16', 'C17', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C28', 'C29']


We count the number of type to see whether the amount is equivalent to the shape of lines in the following graphs: "mass", "h_diameter", "v_diameter" and "brix"

In [201]:
import re

data_type = []
for name in list_dataset["orange_std_col_name"].get_dataframe()["name"].str.upper():
    data_type.append(re.findall("[A-Z]", name)[0])

data_type = pd.DataFrame(data_type, columns=["type"])
data_type["count"] = 1
data_type.groupby(["type"]).count()["count"]

type
A    26
B    30
C    25
Name: count, dtype: int64

In [202]:
list_dataset["orange_std_col_name"].concat_dataframe(data_type["type"])

In [203]:
list_dataset["orange_std_col_name"].get_dataframe().head(10)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286,A
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693,A
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110,A
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458,A
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748,A


## Analyze quantitative data

### Plots for all rows

In [204]:
if GENERATE_PLOTS:
    plot_hist_kde_box_all_columns(list_dataset["orange_std_col_name"])

In [205]:
list_dataset["orange_std_col_name"].get_dataframe().describe()

,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,1.477778,0.406857,222.197531,241.777778,241.358025,11.169259,3.849815,3.024935,7.319445
std,0.297069,0.007092,50.582215,23.652167,22.587225,1.362702,0.774895,0.786520,1.587514
min,0.800000,0.400000,126.000000,195.000000,197.000000,7.400000,2.057520,1.795715,4.076693
25%,1.300000,0.401600,179.000000,222.000000,223.000000,10.400000,3.370111,2.512846,6.148368
50%,1.400000,0.404800,228.000000,242.000000,242.000000,11.100000,3.701077,2.920633,7.332125
75%,1.600000,0.408200,255.000000,257.000000,255.000000,12.300000,4.229802,3.371688,8.199505
max,2.300000,0.423000,357.000000,305.000000,290.000000,14.000000,5.891803,6.609899,11.542480


In [206]:
if GENERATE_PLOTS:
    plot_all_columns_group_by_a_categorical_col(list_dataset["orange_std_col_name"], "type")

#### From above plots, we see that "mass", "h_diameter", "v_diameter" and "brm_ta" plots have clearly patterns corresponding with types.

## Analyze qualitative data

In [207]:
# partitions_dataframe = divide_dataframe_with_categorical_col(list_dataset["orange_std_col_name"], "type")

### Use Welch's t-test to see whether the data could be divided into 3 parts A, B, and C.

In [208]:
# for group in partitions_dataframe.keys():
#     print(f"{group}: {partitions_dataframe[group].shape}")

In [209]:
# partitions_dataframe["A"].head(10)

In [210]:
# columns_to_compare = ["mass", "h_diameter", "v_diameter"]

In [211]:
# calculate_p_values_between_partitions(partitions_dataframe, columns_to_compare)

### All p-values smaller than $ \alpha $ = 0.05
### => Conclusion from Welch's t-test: The dataset could be divided into 3 parts A, B and C

## Matrix plots

### Pairplots

In [212]:
if GENERATE_PLOTS:
    plt.figure(figsize=(16, 16))
    sns.pairplot(list_dataset["orange_std_col_name"].get_dataframe())

#### Some potential features to explain brix are "mass", "h_diameter", "v_diameter", "brix_ta", "brm_ta".

### Heatmap

In [213]:
if GENERATE_PLOTS:
    plt.figure(figsize=(12, 12))
    dataframe = list_dataset["orange_std_col_name"].get_dataframe()
    numeric_cols = list_dataset["orange_std_col_name"].numeric_cols
    dataset_corr = dataframe[numeric_cols].corr()
    sns.heatmap(dataset_corr, annot=True)

#### => Some features that could be used for model: **mass, h_diameter, v_diameter, brix_ta, brm_ta** (quantitative) and **type** (qualitative) to explain **brix**. However, **brix_ta** and **brm_ta** are directly related to **brix**, then they will not be considered.

## Build Models

### Create "ohv_std_col_name" dataframe: ["brix", "mass", "h_diameter", "v_diameter", one-hot-vector-of-type]

In [214]:
ohv_dataset = list_dataset["orange_std_col_name"].get_one_hot_vectorized_dataset("type")
list_dataset["orange_std_col_name_ohv_type"] = ohv_dataset
list_dataset["orange_std_col_name_ohv_type"].get_dataframe().head(10)

orange_std_col_name_ohv_type


,A,B,C,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,True,False,False,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A
1,True,False,False,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A
2,True,False,False,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A
3,True,False,False,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A
4,True,False,False,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A
5,True,False,False,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286,A
6,True,False,False,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693,A
7,True,False,False,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110,A
8,True,False,False,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458,A
9,True,False,False,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748,A


In [215]:
y_cols = ["brix"]
y_variable = list_dataset["orange_std_col_name_ohv_type"].get_dataframe()[y_cols]
y_variable.head(10)

,brix
0,10.9
1,9.8
2,11.0
3,9.0
4,10.9
5,10.7
6,11.2
7,9.2
8,9.5
9,10.5


In [216]:
x_cols = ["mass", "h_diameter", "v_diameter", "B", "C"]
x_variable = list_dataset["orange_std_col_name_ohv_type"].get_dataframe()[x_cols]
x_variable.head(10)

,mass,h_diameter,v_diameter,B,C
0,255,257,262,False,False
1,325,288,278,False,False
2,185,235,237,False,False
3,203,245,245,False,False
4,229,253,249,False,False
5,357,305,290,False,False
6,309,275,285,False,False
7,321,285,280,False,False
8,245,255,259,False,False
9,275,267,270,False,False


### Linear regression models

#### Model train 1: brix $ \sim \beta_0 $ + $ \beta_1 $ * type_a + $ \beta_2 $ * type_b + $ \beta_3 $ * type_c + $ \beta_4 $ * h.diameter + $ \beta_5 $ * v.diameter + $ \beta_6 $ * mass

working

In [217]:
list_dataset.keys()

dict_keys(['orange_std_col_name', 'orange_std_col_name_ohv_type'])

In [218]:
std_dataset_names = ["orange_std_col_name_ohv_type"]

In [219]:
if TRAIN_MODEL_1:

    for dataset_name in std_dataset_names:
        for model_type in ModelType:
            for scaler_type in ScalerType:
                model = Model(
                    model_type=model_type,
                    model_name="model_" + str(model_count),
                    dataset_name="orange_std_col_name_ohv_type",
                    x_cols=x_cols,
                    y_cols=y_cols,
                    dataset=list_dataset[dataset_name],
                    scaler_type=scaler_type,
                )

                result_frame.add_result(model.evaluate())
                model_count += 1

    result_frame.display_result()

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 43.69it/s]


,dataset_name,model_name,model_type,scaler_type,r2,mse
0,orange_std_col_name_ohv_type,model_1,ModelType.LINEAR_REGRESSION,ScalerType.Raw_,0.329090,1.230471
1,orange_std_col_name_ohv_type,model_2,ModelType.LINEAR_REGRESSION,ScalerType.MaxAbsScaler_,0.329090,1.230471
2,orange_std_col_name_ohv_type,model_3,ModelType.LINEAR_REGRESSION,ScalerType.MinMaxScaler_,0.329090,1.230471
4,orange_std_col_name_ohv_type,model_5,ModelType.LINEAR_REGRESSION,ScalerType.RobustScaler_,0.329090,1.230471
5,orange_std_col_name_ohv_type,model_6,ModelType.LINEAR_REGRESSION,ScalerType.StandardScaler_,0.329090,1.230471
9,orange_std_col_name_ohv_type,model_10,ModelType.SVR,ScalerType.Normalizer_,0.309389,1.266603
7,orange_std_col_name_ohv_type,model_8,ModelType.SVR,ScalerType.MaxAbsScaler_,0.295081,1.292844
8,orange_std_col_name_ohv_type,model_9,ModelType.SVR,ScalerType.MinMaxScaler_,0.286888,1.307869
3,orange_std_col_name_ohv_type,model_4,ModelType.LINEAR_REGRESSION,ScalerType.Normalizer_,0.277361,1.325342
11,orange_std_col_name_ohv_type,model_12,ModelType.SVR,ScalerType.StandardScaler_,0.277135,1.325757


In [220]:
result_frame.save_result("result_with_type.xlsx")

Error: CANNOT SAVE FILE.
A file with the same name already exists.
Set overwrite to True to overwrite existed file or change file name!


In [221]:
list_dataset.keys()

dict_keys(['orange_std_col_name', 'orange_std_col_name_ohv_type'])

In [222]:
list_dataset["orange_std_col_name_ohv_type"].get_dataframe()

,A,B,C,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,True,False,False,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A
1,True,False,False,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A
2,True,False,False,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A
3,True,False,False,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A
4,True,False,False,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,False,False,True,C24,1.60,0.4016,177,225,216,12.5,4.115040,3.037638,8.384960,C
77,False,False,True,C25,1.60,0.4016,173,216,220,11.1,4.115040,2.697422,6.984960,C
78,False,False,True,C26,0.95,0.4016,126,195,197,12.8,2.443305,5.238806,10.356695,C
79,False,False,True,C28,1.70,0.4016,181,216,232,11.8,4.372230,2.698852,7.427770,C


In [223]:
x_cols

['mass', 'h_diameter', 'v_diameter', 'B', 'C']

In [224]:
x_cols_no_type = x_cols[:-2]
x_cols_no_type

['mass', 'h_diameter', 'v_diameter']

In [225]:
y_cols

['brix']

In [226]:
list_dataset.keys()

dict_keys(['orange_std_col_name', 'orange_std_col_name_ohv_type'])

In [227]:
list_dataset["orange_std_col_name_ohv_type"].get_dataframe().head(10)

,A,B,C,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,True,False,False,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A
1,True,False,False,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A
2,True,False,False,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A
3,True,False,False,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A
4,True,False,False,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A
5,True,False,False,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286,A
6,True,False,False,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693,A
7,True,False,False,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110,A
8,True,False,False,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458,A
9,True,False,False,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748,A


In [228]:
x_cols_no_type

['mass', 'h_diameter', 'v_diameter']

In [229]:
y_cols

['brix']

#### Model train 2: brix $ \sim \beta_0 $ + $ \beta_1 $ * h.diameter + $ \beta_2 $ * v.diameter + $ \beta_3 $ * mass

In [230]:
if TRAIN_MODEL_2:

    for dataset_name in std_dataset_names:
        for model_type in ModelType:
            for scaler_type in ScalerType:
                model = Model(
                    model_type=model_type,
                    model_name="model_" + str(model_count),
                    dataset_name="orange_std_col_name_no_type",
                    x_cols=x_cols_no_type,
                    y_cols=y_cols,
                    dataset=list_dataset[dataset_name],
                    scaler_type=scaler_type,
                )

                result_frame.add_result(model.evaluate())
                model_count += 1

    result_frame.display_result()

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 55.74it/s]


,dataset_name,model_name,model_type,scaler_type,r2,mse
0,orange_std_col_name_ohv_type,model_1,ModelType.LINEAR_REGRESSION,ScalerType.Raw_,0.329090,1.230471
1,orange_std_col_name_ohv_type,model_2,ModelType.LINEAR_REGRESSION,ScalerType.MaxAbsScaler_,0.329090,1.230471
2,orange_std_col_name_ohv_type,model_3,ModelType.LINEAR_REGRESSION,ScalerType.MinMaxScaler_,0.329090,1.230471
4,orange_std_col_name_ohv_type,model_5,ModelType.LINEAR_REGRESSION,ScalerType.RobustScaler_,0.329090,1.230471
5,orange_std_col_name_ohv_type,model_6,ModelType.LINEAR_REGRESSION,ScalerType.StandardScaler_,0.329090,1.230471
36,orange_std_col_name_no_type,model_37,ModelType.LINEAR_REGRESSION,ScalerType.Raw_,0.317596,1.251551
38,orange_std_col_name_no_type,model_39,ModelType.LINEAR_REGRESSION,ScalerType.MinMaxScaler_,0.317596,1.251551
37,orange_std_col_name_no_type,model_38,ModelType.LINEAR_REGRESSION,ScalerType.MaxAbsScaler_,0.317596,1.251551
40,orange_std_col_name_no_type,model_41,ModelType.LINEAR_REGRESSION,ScalerType.RobustScaler_,0.317596,1.251551
41,orange_std_col_name_no_type,model_42,ModelType.LINEAR_REGRESSION,ScalerType.StandardScaler_,0.317596,1.251551


In [231]:
result_frame.save_result("result_with_and_without_type.xlsx")

Error: CANNOT SAVE FILE.
A file with the same name already exists.
Set overwrite to True to overwrite existed file or change file name!


In [232]:
resnet34_dataframe = pd.read_excel("resnet34.xlsx", sheet_name=0)

In [233]:
resnet34_dataframe.head(20)

,name,resnet34_0,resnet34_1,resnet34_2,resnet34_3,resnet34_4,resnet34_5,resnet34_6,resnet34_7,resnet34_8,...,resnet34_502,resnet34_503,resnet34_504,resnet34_505,resnet34_506,resnet34_507,resnet34_508,resnet34_509,resnet34_510,resnet34_511
0,B12,0.156891,0.135281,0.208339,0.099949,0.606509,0.197465,0.146553,0.190174,0.245006,...,0.139282,0.367176,0.289474,0.158419,0.234442,0.056854,0.174647,0.184782,0.184333,0.193234
1,A2,0.188114,0.145693,0.204161,0.116336,0.527272,0.162754,0.228751,0.213932,0.222092,...,0.203496,0.386706,0.308793,0.202114,0.340296,0.080407,0.177876,0.139048,0.176140,0.208788
2,B8,0.173824,0.126600,0.278486,0.195133,0.489945,0.178314,0.230793,0.169339,0.302398,...,0.312228,0.415683,0.320124,0.285096,0.289027,0.085196,0.150526,0.240075,0.189648,0.225144
3,C26,0.144006,0.098398,0.209701,0.219367,0.548297,0.212151,0.215506,0.139116,0.276130,...,0.456208,0.386990,0.322745,0.301975,0.240051,0.098982,0.157585,0.218558,0.173098,0.214293
4,C25,0.184470,0.138784,0.198218,0.179938,0.495524,0.243386,0.167743,0.181745,0.312281,...,0.263740,0.334849,0.350409,0.259369,0.327031,0.074745,0.210896,0.169288,0.178689,0.188826
5,C29,0.164305,0.117410,0.224094,0.195399,0.499424,0.194218,0.168049,0.201609,0.289058,...,0.389878,0.330334,0.338783,0.196602,0.282925,0.059246,0.222848,0.165042,0.178285,0.202234
6,C5,0.143685,0.126570,0.210499,0.123183,0.616666,0.208228,0.166801,0.205509,0.288818,...,0.239749,0.346334,0.317840,0.185028,0.234601,0.038626,0.270342,0.174342,0.181206,0.200992
7,C13,0.205536,0.127269,0.201759,0.163079,0.510231,0.228772,0.177624,0.203865,0.253171,...,0.211222,0.376262,0.340810,0.213568,0.265492,0.071630,0.237319,0.171994,0.173610,0.218804
8,B5,0.153832,0.122729,0.139407,0.085343,0.816709,0.156730,0.140327,0.186524,0.202721,...,0.113562,0.403603,0.223014,0.132067,0.166786,0.027575,0.223429,0.147432,0.096567,0.150655
9,A30,0.215418,0.147183,0.180083,0.147134,0.577130,0.138726,0.201425,0.208275,0.270178,...,0.203487,0.372896,0.283954,0.217919,0.312682,0.074968,0.174066,0.203316,0.143396,0.187734


In [234]:
resnet34_dataframe["name"] = resnet34_dataframe["name"].astype(str)

In [235]:
list_dataset["orange_std_col_name_ohv_type"].get_dataframe()["name"].astype(str)

0      A1
1      A2
2      A3
3      A4
4      A5
     ... 
76    C24
77    C25
78    C26
79    C28
80    C29
Name: name, Length: 81, dtype: object

In [236]:
orange_std_col_name_ohv_type_resnet34 = list_dataset["orange_std_col_name_ohv_type"].get_dataframe().merge(resnet34_dataframe, how="inner", on="name")

In [237]:
list_dataset["orange_std_col_name_ohv_type_resnet34"] = Dataset("orange_std_col_name_ohv_type_resnet34", orange_std_col_name_ohv_type_resnet34, ScalerType.Raw_)

In [248]:
list_dataset["orange_std_col_name_ohv_type_resnet34"].get_dataframe()

,A,B,C,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,...,resnet34_502,resnet34_503,resnet34_504,resnet34_505,resnet34_506,resnet34_507,resnet34_508,resnet34_509,resnet34_510,resnet34_511
0,True,False,False,A1,1.90,0.4000,255,257,262,10.9,...,0.300284,0.347631,0.272064,0.255078,0.361392,0.055421,0.178558,0.166191,0.152494,0.186288
1,True,False,False,A2,1.20,0.4000,325,288,278,9.8,...,0.203496,0.386706,0.308793,0.202114,0.340296,0.080407,0.177876,0.139048,0.176140,0.208788
2,True,False,False,A3,2.30,0.4000,185,235,237,11.0,...,0.139981,0.375960,0.257595,0.138591,0.206467,0.035784,0.203072,0.160679,0.104960,0.161872
3,True,False,False,A4,1.40,0.4128,203,245,245,9.0,...,0.162073,0.390348,0.323551,0.167936,0.239144,0.043754,0.196060,0.178945,0.150613,0.179786
4,True,False,False,A5,1.20,0.4000,229,253,249,10.9,...,0.222409,0.384862,0.263854,0.226789,0.318150,0.061865,0.224037,0.219195,0.199771,0.213477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,False,False,True,C24,1.60,0.4016,177,225,216,12.5,...,0.307272,0.344841,0.430694,0.305798,0.394726,0.072334,0.213526,0.195215,0.203654,0.183547
77,False,False,True,C25,1.60,0.4016,173,216,220,11.1,...,0.263740,0.334849,0.350409,0.259369,0.327031,0.074745,0.210896,0.169288,0.178689,0.188826
78,False,False,True,C26,0.95,0.4016,126,195,197,12.8,...,0.456208,0.386990,0.322745,0.301975,0.240051,0.098982,0.157585,0.218558,0.173098,0.214293
79,False,False,True,C28,1.70,0.4016,181,216,232,11.8,...,0.259605,0.352100,0.380756,0.279616,0.324362,0.088129,0.189930,0.184220,0.166675,0.205620


In [251]:
list_dataset["orange_std_col_name_ohv_type_resnet34"].get_dataframe().iloc[:,:20]

,A,B,C,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type,resnet34_0,resnet34_1,resnet34_2,resnet34_3,resnet34_4,resnet34_5
0,True,False,False,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A,0.140042,0.165523,0.206252,0.121150,0.611558,0.251576
1,True,False,False,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A,0.188114,0.145693,0.204161,0.116336,0.527272,0.162754
2,True,False,False,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A,0.170248,0.143860,0.155040,0.102307,0.824027,0.183213
3,True,False,False,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A,0.155330,0.146644,0.179467,0.128099,0.713304,0.201185
4,True,False,False,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A,0.170064,0.135311,0.230836,0.131059,0.544553,0.206472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,False,False,True,C24,1.60,0.4016,177,225,216,12.5,4.115040,3.037638,8.384960,C,0.184246,0.135858,0.192986,0.189380,0.459203,0.224478
77,False,False,True,C25,1.60,0.4016,173,216,220,11.1,4.115040,2.697422,6.984960,C,0.184470,0.138784,0.198218,0.179938,0.495524,0.243386
78,False,False,True,C26,0.95,0.4016,126,195,197,12.8,2.443305,5.238806,10.356695,C,0.144006,0.098398,0.209701,0.219367,0.548297,0.212151
79,False,False,True,C28,1.70,0.4016,181,216,232,11.8,4.372230,2.698852,7.427770,C,0.190891,0.143164,0.186174,0.153178,0.483471,0.237820


In [285]:
number_of_cols = len(list_dataset["orange_std_col_name_ohv_type_resnet34"].get_dataframe().columns)
x_cols_resnet34 = list(list_dataset["orange_std_col_name_ohv_type_resnet34"].get_dataframe().iloc[:,np.r_[1,2,6:9,14:number_of_cols]].columns)
x_cols_resnet34

['B',
 'C',
 'mass',
 'h_diameter',
 'v_diameter',
 'resnet34_0',
 'resnet34_1',
 'resnet34_2',
 'resnet34_3',
 'resnet34_4',
 'resnet34_5',
 'resnet34_6',
 'resnet34_7',
 'resnet34_8',
 'resnet34_9',
 'resnet34_10',
 'resnet34_11',
 'resnet34_12',
 'resnet34_13',
 'resnet34_14',
 'resnet34_15',
 'resnet34_16',
 'resnet34_17',
 'resnet34_18',
 'resnet34_19',
 'resnet34_20',
 'resnet34_21',
 'resnet34_22',
 'resnet34_23',
 'resnet34_24',
 'resnet34_25',
 'resnet34_26',
 'resnet34_27',
 'resnet34_28',
 'resnet34_29',
 'resnet34_30',
 'resnet34_31',
 'resnet34_32',
 'resnet34_33',
 'resnet34_34',
 'resnet34_35',
 'resnet34_36',
 'resnet34_37',
 'resnet34_38',
 'resnet34_39',
 'resnet34_40',
 'resnet34_41',
 'resnet34_42',
 'resnet34_43',
 'resnet34_44',
 'resnet34_45',
 'resnet34_46',
 'resnet34_47',
 'resnet34_48',
 'resnet34_49',
 'resnet34_50',
 'resnet34_51',
 'resnet34_52',
 'resnet34_53',
 'resnet34_54',
 'resnet34_55',
 'resnet34_56',
 'resnet34_57',
 'resnet34_58',
 'resnet34_59',

In [286]:
y_cols_resnet34 = ["brix"]

#### Model train 3: brix $ \sim \beta_0 $ + $ \beta_1 $ * type_a + $ \beta_2 $ * type_b + $ \beta_3 $ * type_c + $ \beta_4 $ * h.diameter + $ \beta_5 $ * v.diameter + $ \beta_6 $ * mass + $ \beta_7 $ * resnet34_0 + ... + $ \beta_p $ * resnet34_p

In [ ]:
if TRAIN_MODEL_3:

    for dataset_name in std_dataset_names:
        for model_type in ModelType:
            for scaler_type in ScalerType:
                model = Model(
                    model_type=model_type,
                    model_name="model_" + str(model_count),
                    dataset_name="orange_std_col_name_ohv_type_resnet34",
                    x_cols=x_cols_resnet34,
                    y_cols=y_cols_resnet34,
                    dataset=list_dataset[dataset_name],
                    scaler_type=scaler_type,
                )

                result_frame.add_result(model.evaluate())
                model_count += 1

    result_frame.display_result()

In [301]:
model = Model(
    model_type=ModelType.LIGHT_GBM,
    model_name="model_" + str(model_count),
    dataset_name="orange_std_col_name_ohv_type_resnet34",
    x_cols=x_cols_resnet34,
    y_cols=y_cols_resnet34,
    dataset=list_dataset["orange_std_col_name_ohv_type_resnet34"],
    scaler_type=ScalerType.Normalizer_,
)

temp_result_frame = ResultFrame()
temp_result_frame.add_result(model.evaluate())
temp_result_frame.display_result()

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:05<00:00, 14.63it/s]


,dataset_name,model_name,model_type,scaler_type,r2,mse
0,orange_std_col_name_ohv_type_resnet34,model_73,ModelType.LIGHT_GBM,ScalerType.Normalizer_,0.290612,1.301039
